In [1]:
import re
import glob
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor
from threading import Thread
from multiprocessing import Pool

import pytesseract
from PIL import Image

In [2]:
im_paths = glob.glob( './datasets/test_pdf_imgs/*.png')


def time_it(func):
    
    def wrapper():
        start_ = time.time()
        res = func()
        print(f'{func.__name__} 耗时{time.time() - start_ }秒')
        return res
    return wrapper


def img_2_text(im_path, lang='chi_sim+eng'):
    page = int(re.findall(r'第(\d+)页', im_path)[0])
    im = Image.open(im_path)
    text =  pytesseract.image_to_string(im, lang=lang)
    with open(f'./datasets/test_pdf_imgs/第{page}页.txt', 'w') as f:
        f.write(''.join(text.split(' ')))

## 普通版

In [3]:
@time_it
def common_version():
    for im_path in im_paths:
        img_2_text(im_path)

common_version()

common_version 耗时71.91380310058594秒


## 多线程版

In [4]:
@time_it
def theading_version():
    with ThreadPoolExecutor(4) as executor:
        executor.map(img_2_text, im_paths)

theading_version()

theading_version 耗时35.4434289932251秒


## 协程版
`pip install ipython ipykernel --upgrade` fix `RuntimeError: asyncio.run() cannot be called from a running event loop`

In [5]:
# 可能哪弄错了，不理想

## 多进程版

In [6]:
@time_it
def process_version():
    pool = Pool(4)
    pool.map(img_2_text, im_paths)

process_version()

process_version 耗时34.12662601470947秒


不应该啊，pytesseract.image_to_string应该是个 CPU 密集型啊